In [2]:
import os,sys
import argparse
import numpy as np
import uproot
import awkward as ak
import pandas as pd
from time import time

import torch
from torch_geometric.data import Data
import pickle
import json

import warnings
warnings.filterwarnings("ignore")

sys.path.append('../')
from utils.pltutils import plt

# import input range
with open('../utils/hgcal_input_range.json','r') as f:
        input_range = json.load(f)

In [4]:
def torchify(feat, graph_x = None):
    data = [Data(x = torch.from_numpy(ak.to_numpy(ele).astype(np.float32))) for ele in feat]
    if graph_x is not None:
        for d, gx in zip(data, graph_x):
            d.graph_x = gx
    return data

def rescale(feature, minval, maxval):
    top = feature-minval
    bot = maxval-minval
    return top/bot

def cartfeat_HGCAL(x, y, z, En):
    E = rescale(En, input_range['HGCAL_Min'], input_range['HGCAL_Max'])
    x = rescale(x, input_range['HGCAL_X_Min'], input_range['HGCAL_X_Max'])
    y = rescale(y, input_range['HGCAL_Y_Min'], input_range['HGCAL_Y_Max'])
    z = rescale(z, input_range['HGCAL_Z_Min'], input_range['HGCAL_Z_Max'])
    return ak.concatenate((x[:,:,None], y[:,:,None], z[:,:,None], E[:,:,None]), -1)

In [3]:
path_to_input = '/home/rusack/shared/nTuples/hgcal_electrons/data/ntuple_455.root'
input_type = 'file'
output_folder = '/home/rusack/shared/pickles/hgcal_electron/data_h3_selection/20'
data_type = 'data'

In [7]:
filelist = []

if input_type=='file':
    filelist.append(path_to_input)
elif input_type=='folder':
    filelist = [ '{}/{}'.format(path_to_input,f.replace('\\','')) for f in os.listdir(path_to_input) if '.root' in f ]
else:
    print('Invalid file type!')
    exit(2)

# check if files exist
for f in filelist:
    if not os.path.exists(f):
        print(f)
        print('File not found!')
        exit(2)

if data_type not in ['data','mc']:
    print('Check your data type!')
    exit(2)

event_file_map = {}
for e in [20, 30, 50, 80, 100, 120, 150, 200, 250, 300]:
    DTYPE = 'DATA'
    if data_type=='mc': DTYPE='MC'
    with open('../HGCALTB_events/{}_EvtList_{}GeV.txt'.format(DTYPE,e),'r') as f:
        lines = f.readlines()

    filename = ''
    for l in lines:
        l = l.strip('\n')
        if 'Runs' in l: continue
        if 'eos' in l:
            filename = l.split('/')[-1]
            event_file_map[filename] = []
        else:
            x = l.split('\t\t ')
            event_file_map[filename].append(np.array([int(x[0]), int(x[1])]))
    for f in event_file_map.keys(): event_file_map[f] = np.array(event_file_map[f])

hit_list = ['event', 'run', 'rechit_x', 'rechit_y', 'rechit_z', 'rechit_layer',
        'rechit_energy', 'rechit_chip', 'rechit_channel', 'trueBeamEnergy']
ip_list = ['event', 'ntracks', 'dwcReferenceType', 'b_x',
        'b_y', 'm_x', 'm_y', 'trackChi2_X', 'trackChi2_Y']

In [58]:
pickles = {}
data_rechit_tree = None
data_ip_tree = None
data_hit_map = None
data_ip_map = None
for ifile, f in enumerate(filelist):
    print("Reading file {}".format(f))
    er = event_file_map[f.split('/')[-1]]
    with uproot.open(f) as data_tree:

        data_rechit_tree = data_tree['rechitntupler/hits']
        data_ip_tree = data_tree['trackimpactntupler/impactPoints']
        data_hit_map = data_rechit_tree.arrays(hit_list)
        data_ip_map = data_ip_tree.arrays(ip_list)

        event_check = [ x in er[:,0] for x in data_hit_map.event ]
        run_check = [ x in er[:,0] for x in data_hit_map.event ]
        preselection = ak.Array(event_check)*ak.Array(run_check)
        data_hit_map['all_target'] = data_hit_map.trueBeamEnergy

        channel_mask = ~((data_hit_map.rechit_chip==0)&(data_hit_map.rechit_layer==1))
        channel_mask = (~((data_hit_map.rechit_layer==3)&(data_hit_map.rechit_channel==22)))*channel_mask

        noise_th = data_hit_map.rechit_energy>=0.5
        hit_selection = preselection*noise_th*channel_mask

        quantities = ['rechit_x', 'rechit_y', 'rechit_z', 'rechit_energy', 'all_target']

        for q_ in quantities:
            print(q_)
            if ifile==0:
                if q_=='all_target':
                    pickles[q_] = data_hit_map[q_][preselection]   
                else:
                    pickles[q_] = data_hit_map[q_][hit_selection]
            else:
                if q_=='all_target':
                    pickles[q_] = np.concatenate((data_hit_map[q_][preselection], pickles[q_]), axis=0)
                else:
                    pickles[q_] = np.concatenate((data_hit_map[q_][hit_selection], pickles[q_]), axis=0)

Reading file /home/rusack/shared/nTuples/hgcal_electrons/data/ntuple_455.root
rechit_x
rechit_y
rechit_z
rechit_energy
all_target


In [59]:
L = data_hit_map['rechit_layer'][preselection*noise_th*(~channel_mask)]
C = data_hit_map['rechit_chip'][preselection*noise_th*(~channel_mask)]
E = data_hit_map['rechit_energy'][preselection*noise_th*(~channel_mask)]

In [63]:
np.mean(ak.flatten(C[L==3]))

1.495049504950495

In [64]:
pickles['rechit_energy'][1]

<Array [1.2, 6.48, 0.855, ... 1.31, 1.17] type='296 * float32'>

In [31]:
arr = cartfeat_HGCAL(pickles['rechit_x'],
               pickles['rechit_y'],
               pickles['rechit_z'],
               pickles['rechit_energy'])

pickles['cartfeat'] = torchify(arr)

print('Dumping pickle files...')
path_to_output = output_folder

for q_ in pickles:
    output_file = '{}/{}.pickle'.format(path_to_output, q_)
    with open(output_file, 'wb') as f_:
        t0=time()
        if q_=='cartfeat':
            torch.save(pickles[q_], f_, pickle_protocol=4)
        else:
            pickle.dump(pickles[q_], f_, protocol=4)
        print('File {}.pickle dumped in {} s'.format(q_, time()-t0))

Dumping pickle files...
File rechit_x.pickle dumped in 0.0026404857635498047 s
File rechit_y.pickle dumped in 0.0024149417877197266 s
File rechit_z.pickle dumped in 0.002569437026977539 s
File rechit_energy.pickle dumped in 0.002742767333984375 s
File all_target.pickle dumped in 0.0002944469451904297 s
File cartfeat.pickle dumped in 0.4558699131011963 s
